<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#9.1-复杂数据的局部性建模" data-toc-modified-id="9.1-复杂数据的局部性建模-1">9.1 复杂数据的局部性建模</a></span></li><li><span><a href="#9.2-连续型和离散型特征的树的模型的构建" data-toc-modified-id="9.2-连续型和离散型特征的树的模型的构建-2">9.2 连续型和离散型特征的树的模型的构建</a></span></li><li><span><a href="#9.3-将CART算法用于回归" data-toc-modified-id="9.3-将CART算法用于回归-3">9.3 将CART算法用于回归</a></span></li><li><span><a href="#9.3.1-构建树" data-toc-modified-id="9.3.1-构建树-4">9.3.1 构建树</a></span><ul class="toc-item"><li><span><a href="#9.3.2-运行代码" data-toc-modified-id="9.3.2-运行代码-4.1">9.3.2 运行代码</a></span></li></ul></li></ul></div>

# 树回归
现实生活中，很多问题都是非线性的，不可能使用全局现行模型来拟合任何数据，一种可行的方法是将数据集切分成很多块易建模的数据，然后利用前一章的线性回归技术来建模。

## 9.1 复杂数据的局部性建模
决策树不断地将数据切分成小的数据集，直到所有目标变量完全相同，或者数据数据不能再切分为止，决策树是一种贪心算法，它要在给定时间内作出最佳选择，但并不关心能否达到全局最优。

- 优点：可以对复杂和非线性的数据建模
- 缺点：结果不易理解
- 适用数据类型：数值型和标称型数据

二元切分法：每次把数据集切分成两份，如果数据的某特征值等于切分所要求的值，那么这些数据就进入树的左子树，反之则进入树的右子树。

二元切分法易于对树构建过程进行调整以处理连续型特征，具体处理方法是：如果特征值大于给定值就走左子树，否则就走右子树。CART稍作修改能够处理回归问题。

回归树和分类树的思路类似，但叶节点的数据类型不是离散型，而是连续型。

1. 收集数据：采用任意方法收集数据
2. 准备数据：需要数值型的数据，标称型的数据应该映射成二值型数据
3. 分析数据：绘出数据的二维可视化显示结果，以字典方式生成树
4. 训练算法：大部分时间都花费在叶节点树模型构建上
5. 测试算法：使用测试数据集上的$R^2$值来分析模型效果
6. 使用模型：使用训练出的树做预测

## 9.2 连续型和离散型特征的树的模型的构建

1. 使用字典存储树的结构
2. 面向对象构造树节点
```python
class treeNode():
    def __init__(self, feat, val, right, left):
        featureToSplitOn = feat
        valueOfSplit = val
        rightBranch = right
        leftBranch = left
```
createTree()伪代码：

    找到最佳待切分特征：
        如果该节点不能再分，将该节点存为叶节点
        执行二元切分
        在右子树调用createTree()方法
        在左子树调用createTree()方法

In [188]:
# CART算法
from numpy import *
def loadDataSet(fileName):
    dataMat = []
    fr = open(fileName)
    for line in fr.readlines():
        curLine = line.strip().split('\t')
        fltLine = map(float, curLine)    # 将每行映射成浮点数
        dataMat.append(fltLine)
    return dataMat
# utils
def regLeaf(dataSet):#returns the value used for each leaf
    return mean(dataSet[:,-1])

def regErr(dataSet):
    return var(dataSet[:,-1]) * shape(dataSet)[0]

def binSplitDataSet(dataSet, feature, value):

    mat0 = dataSet[nonzero(dataSet[:,feature] > value)[0], :]
    mat1 = dataSet[nonzero(dataSet[:,feature] <= value)[0], :]
    return mat0, mat1

def createTree(dataSet, leafType=regLeaf, errType=regErr, ops=(1, 4)):
    feat, val = chooseBestSplit(dataSet, leafType, errType, ops)
    if feat == None:    # 满足停止条件时返回叶节点值
        return val
    retTree = {}
    retTree['spInd'] = feat
    retTree['spVal'] = val
    lSet, rSet = binSplitDataSet(dataSet, feat, val)
    print lSet
    retTree['left'] = createTree(lSet, leafType, errType, ops)
    retTree['right'] = createTree(rSet, leafType, errType, ops)
    return retTree, lSet
        

In [189]:
testMat = mat((eye(4)))
mat0, mat1 = binSplitDataSet(testMat, 1, 0.5)

## 9.3 将CART算法用于回归
为了成功构建以分段常数为叶节点的树，需要度量出数据的一致性。
## 9.3.1 构建树
chooseBestSplit()——给定某个误差计算方法，该函数会找到数据集上最佳二元切分方式。
该函数的为代码大致如下：

    对每个特征：
        对每个特征：
            将数据集分为两个部分
            计算切分误差
            如果当前误差小于最小误差，那么将当前且分设置为最佳切分，并更新最小误差
    返回最佳切分特征和阈值

In [190]:
# 回归树切分函数
def chooseBestSplit(dataSet, leafType=regLeaf, errType=regErr, ops=(1,4)):
    tolS = ops[0]
    tolN = ops[1]
    # 如果所有值相等，则退出
    if len(set(dataSet[:,-1].T.tolist()[0])) == 1:
        return None, leafType(dataSet)
    m, n = shape(dataSet)
    S = errType(dataSet)
    bestS, bestIndex, bestValue = inf, 0, 0
    for featIndex in range(n-1):
        for splitVal in set(dataSet[:,featIndex].flat):
            mat0, mat1 = binSplitDataSet(dataSet, featIndex, splitVal)
            if (shape(mat0)[0] < tolN) or (shape(mat1)[0] < tolN):
                continue
            newS = errType(mat0) + errType(mat1)
            if newS < bestS:
                bestIndex = featIndex
                bestValue = splitVal
                bestS = newS
    # 如果误差减少不大，则退出
    if abs(S - bestS) < tolS:
        return None, leafType(dataSet)
    mat0, mat1 = binSplitDataSet(dataSet, bestIndex, bestValue)
    # 如果切分出来的数据很小，则退出
    if (shape(mat0)[0] < tolN) or (shape(mat1)[0] < tolN):
        return None, leafType(dataSet)
#     print bestIndex, bestValue
    return bestIndex, bestValue

### 9.3.2 运行代码


In [199]:
myDat = loadDataSet('ex00.txt')
myMat = mat(myDat)
printInf, lset = createTree(myMat)
# print myMat
lsetC = lset.copy()
# print printInf

[[ 0.993349  1.077553]
 [ 0.530897  0.893462]
 [ 0.712386  0.564858]
 [ 0.691115  0.834391]
 [ 0.727098  1.000567]
 [ 0.951949  0.945255]
 [ 0.768596  0.760219]
 [ 0.541314  0.893748]
 [ 0.673195  0.915077]
 [ 0.517921  1.493586]
 [ 0.703755  1.101678]
 [ 0.837522  1.229373]
 [ 0.919384  1.029889]
 [ 0.814825  1.095206]
 [ 0.61127   0.982036]
 [ 0.869077  1.114825]
 [ 0.620599  1.334421]
 [ 0.820802  1.325907]
 [ 0.520044  0.961983]
 [ 0.819823  0.835264]
 [ 0.975022  0.673579]
 [ 0.953112  1.06469 ]
 [ 0.804586  0.924033]
 [ 0.625336  0.623696]
 [ 0.656218  0.958506]
 [ 0.834078  1.01058 ]
 [ 0.78193   1.074488]
 [ 0.678287  0.907727]
 [ 0.996757  1.035533]
 [ 0.59021   1.336661]
 [ 0.561362  1.070529]
 [ 0.539558  1.053846]
 [ 0.79024   0.533214]
 [ 0.89593   1.077275]
 [ 0.559763  1.253151]
 [ 0.643665  1.024241]
 [ 0.877241  0.797005]
 [ 0.613765  1.621091]
 [ 0.645762  1.026886]
 [ 0.651376  1.315384]
 [ 0.697718  1.212434]
 [ 0.742527  1.087056]
 [ 0.901056  1.0559  ]
 [ 0.948268

ValueError: too many values to unpack

In [201]:
index, val = chooseBestSplit(lsetC, leafType=regLeaf, errType=regErr, ops=(1,4))

NameError: name 'lsetC' is not defined

In [202]:
print lset

NameError: name 'lset' is not defined